# Azores high: analysis

__The goal__:  We're going to (loosely) reproduce results from Cresswell-Clay et al. (2022)$^1$ from scratch, using data stored on WHOI's CMIP server. 

__What did this study do?__  Cresswell-Clay et al. compared the observed expansion of the Azores High over the last $\sim$100 years with its variability on longer timescales. The central question is whether this expansion can be attributed to external forcing (e.g., greenhouse gas emissions) or could be explained by natural climate variability.

To address this question, Cresswell-Clay et al. compare the magnitude of the Azores High's recent expansion to its simulated variability in an ensemble of $\sim$1,200 year simulations from a global climate model (the Community Earth System Model Last-Millenium Ensemble, or CESM-LME). They also compare the model's reconstruction of Azores High extent over this period to a paleoclimate "proxy" (a $\sim$1200-year record of $\delta^{13}C$ obtained from a stalagmite in a Portuguese cave)$^2$. As suggested in the paper's title, the authors find the recent expansion is "unprecedented" and very unlikely to occur from natural climate variability.

__What is the Azores High?__ A "local maximum" of surface air pressure, located approximately over the Azores Islands (about 1,500 km west of Portugal, in the North Atlantic Ocean) during the winter months. 

__Why does it expansion matter?__ One reason: high pressure is associated with drier conditions (e.g., less rain). If the high pressure region expands, we might expect regions in the "expansion zone" to become drier.

__What are we going to do?__ We're going to reproduce versions of Figs. 1, 2c, and 3d-e, from scratch (the authors' original code is also available online$^3$). The goal is to provide a practical example of how to use reanalysis and climate model output to answer questions about climate. 

__References__  
$^1$Cresswell-Clay, N. et al. (2022). "Twentieth-century Azores High expansion unprecedented in the past 1,200 years". *Nat. Geosci.* 15, 548–553.  
$^2$Thatcher, D. L. et al. (2023) "Iberian hydroclimate variability and the Azores High during the last 1200 years: evidence from proxy records and climate model simulations". *Clim Dyn* 60, 2365–2387.  
$^3$https://github.com/nathanielcresswellclay/AzoresHighExpansion/tree/main

## Outline
This example is divided into two parts:
1. __"Observed" expansion and model validation__: we'll plot the "observed" expansion of the Azores High – based on reanalysis output – and compare it to the expansion simulated by the CESM-LME over the overlapping period (approximately 1850-2005). The motivating question is: can we trust the model over the period where it *doesn't* overlap with observations?
2. __Analysis of model output and comparison to cave record__: we'll compare statistics between CESM-LME ensembles with different "external forcing". In particular, we'll look at four types of forcing: (a) greenhouse gas emissions, (b) orbital/solar variability, (c) volcanic activity, and (d) all of the above (denoted "Full" in the paper). We'll also compare the Azores High variability "Full" forcing scenario to the $\delta^{13}C$ variability in the Portuguese stalagmite.

## Packages

In [ ]:
import xarray as xr
import numpy as np
from tqdm import tqdm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import pathlib

## Set plotting defaults
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## Load data

In [ ]:
def load_AHA(save_fp):
    """Load AHA data from given directory"""

    ## names of datasets
    names = ["era", "noaa", "lme_full", "lme_orb", "lme_volc", "lme_GHG"]

    ## function to get absolute path to dataset
    get_fp = lambda name: pathlib.Path(AHA_save_fp, f"AHA_{name}.nc")

    ## function to open single dataset and rename it
    open_single = lambda name: xr.open_dataarray(get_fp(name)).rename(name)

    return xr.merge([open_single(name) for name in names])


## where is the data saved?
AHA_save_fp = pathlib.Path("data/AHA")

## Load the data
aha = load_AHA(AHA_save_fp)

### LME model validation (Fig 2c)
Metric: 25-year rolling count of AHA extremes

In [ ]:
def count_extremes(AHA, cutoff_perc=90.0, window=25):
    """Get rolling count of Azores High extreme events.
    Args:
    - cutoff_perc is percentile value in range (0 and 100) used to define
        'extreme' events
    - window is an integer specifying how many years the rolling window is.
    """

    ## get threshold for extreme events
    threshold = AHA.quantile(q=cutoff_perc / 100)

    ## Get boolean array: True if AHA exceeds thresh
    exceeds_thresh = AHA > threshold

    ## Get rolling count
    rolling_kwargs = dict(dim={"year": window}, center=True, min_periods=10)
    rolling_count = exceeds_thresh.rolling(**rolling_kwargs).mean() * window

    return rolling_count

In [ ]:
## count extremes (from 1850-2005)
extreme_count = count_extremes(aha.sel(year=slice(1850, 2005)))

## make plot
fig, ax = plt.subplots(figsize=(6, 3))

## plot reanalysis
ax.plot(extreme_count.year, extreme_count["noaa"], label="NOAA", c="purple")
ax.plot(extreme_count.year, extreme_count["era"], label="ERA", c="blue")

# ## plot LME mean and min/max range
ax.plot(
    extreme_count.year,
    extreme_count["lme_full"].mean("ensemble_member"),
    label="LME",
    c="orange",
)
ax.plot(
    extreme_count.year,
    extreme_count["lme_full"].max("ensemble_member"),
    c="orange",
    lw=0.5,
)
ax.plot(
    extreme_count.year,
    extreme_count["lme_full"].min("ensemble_member"),
    c="orange",
    lw=0.5,
)

# ax.plot(rolling_count.year, rolling_count["era"])

## label plot
ax.legend()
ax.set_xlabel("Year")
ax.set_ylabel("Count (25-yr rolling)")
plt.show()

### LME single-forcing comparison (Fig 3c,d)

#### Functions for loading paleo data and binning by century

In [ ]:
def load_paleo_data():
    """
    Function to load Paleo data. Description of data here:
    https://www.ncei.noaa.gov/access/paleo-search/study/37160
    """

    ## data is located at following link:
    fp = r"https://www.ncei.noaa.gov/pub/data/paleo/speleothem/europe/portugal/thatcher2022/thatcher2022-composite_isotope_records.txt"

    ## open dataset using Pandas
    paleo_data = pd.read_csv(fp, skiprows=117, sep=r"\s+").set_index("age_CE")

    ## convert to xarray and rename coordinate "year"
    paleo_data = paleo_data.to_xarray().rename({"age_CE": "year"})

    ## switch order of year coordinate so that it's increasing
    paleo_data = paleo_data.reindex({"year": paleo_data.year.values[::-1]})

    return paleo_data


def floor_nearest100(x):
    """rounds down to nearest 100"""

    return np.floor(x / 100) * 100


def bin_by_century(data, bin_offset=0):
    """
    Function to bin data by century. Args:
    - 'bin_offset' is number in [0, 100) representing
        where the first bin begins.
    """

    ## Get century to label data with
    century = floor_nearest100(data["year"] - bin_offset) + bin_offset

    ## add century as new coordinate on data
    ## (this makes it easy to group data by century)
    data = data.assign_coords({"century": century.astype(int)})

    # get mean for each century
    data_binned = data.groupby("century").mean()

    return data_binned

#### Compute stats

In [ ]:
## Load paleo data
paleo_data = load_paleo_data()

## Get count of extremes on full LME simulation (100-year window)
extreme_count = count_extremes(
    aha.drop_vars(["era", "noaa"]), window=100, cutoff_perc=90
)

## Compute binned average for paleo data
offset = 25  # the first bin begins at year (800 + offset)
paleo_data_binned = bin_by_century(paleo_data, bin_offset=offset)

## compute binned average for AHA ensemble mean
aha_mean_binned = bin_by_century(aha.mean("ensemble_member"), bin_offset=offset)

## combined binned data into single DataArray
data_binned = xr.merge([paleo_data_binned, aha_mean_binned])

#### Plot results in style of paper
Plotting functions:

In [ ]:
def plot_LME_extremes(ax):
    """function to plot LME extremes on ax object"""

    ## specify colors and linestyles for each LME run
    labels = ["Full", "GHG", "Volcanic", "Solar"]
    colors = ["k", "green", "red", "orange"]
    scales = [1, 0.75, 0.5, 0.5]

    ## Plot lines one-by-one
    for label, color, scale in zip(labels, colors, scales):

        ## get linewidth based on scale
        mean_width = 2.0 * scale
        range_width = 0.5 * scale

        ## plot ensemble mean
        ax.plot(
            ensemble_mean.year,
            ensemble_mean[label],
            label=label,
            c=color,
            lw=mean_width,
        )

    ## plot standard deviation for "Full" run
    upper_bound = ensemble_mean["Full"] + ensemble_std["Full"]
    lower_bound = ensemble_mean["Full"] - ensemble_std["Full"]
    ax.fill_between(ensemble_mean.year, upper_bound, lower_bound, color="k", alpha=0.07)

    ## label plot
    ax.legend(prop={"size": 8})
    ax.set_xlabel("Year")
    ax.set_ylabel("Extreme events")
    ax.set_yticks([5, 10, 15])
    ax.set_ylim([None, 17.5])

    return ax


def plot_paleo_comparison(ax, data_binned):
    """function to plot paleo data and AHA over time"""

    ## get edges of bins (used for plotting)
    bin_starts = data_binned.century.values
    bin_edges = np.insert(bin_starts, obj=len(bin_starts), values=bin_starts[-1] + 100)

    ## get color for paleo plot
    color_paleo = sns.color_palette()[0]

    ## plot paleo data first
    paleo_plot = ax.stairs(data_binned["d13C_compSuess"], bin_edges, color=color_paleo)
    ax.set_ylabel(r"$\delta^{13}C$", c=color_paleo)
    ax.set_yticks([-1, -2, -3], labels=[-1, -2, -3], color=color_paleo)
    ax.set_xlim([830, 2020])

    ## plot AHA on same x-axis
    ax1 = ax.twinx()
    ax1.stairs(data_binned["lme_full"], bin_edges, color="k")
    ax1.set_ylabel(r"Azores area ($km^2$)")
    ax1.set_yticks([9e6, 11e6, 13e6])
    ax1.set_ylim([9e6, 14e6])

    return ax, ax1

Make plot:

In [ ]:
fig = plt.figure(figsize=(7, 5))
gs = mpl.gridspec.GridSpec(2, 1, height_ratios=[0.7, 1])

## plot binned paleo data and AHA over time
ax0 = fig.add_subplot(gs[0])
ax00, ax01 = plot_paleo_comparison(ax0, data_binned)
ax00.set_xticks([])

## Plot extreme count over time
# ax1 = fig.add_subplot(gs[1])
# ax1 = plot_LME_extremes(ax1)

# ## make sure x-axes match
# ax1.set_xlim(ax0.get_xlim())

plt.show()

# Spatial plots

#### Functions to compute and plot composite

In [ ]:
def make_composite(data, year_range0=[1950, 1979], year_range1=[1980, 2007]):
    """get composite, defined as difference between data
    when averaged over year_range1 and year_range0.
    """

    ## compute means
    mean1 = data.sel(year=slice(*year_range1)).mean("year")
    mean0 = data.sel(year=slice(*year_range0)).mean("year")

    return mean1 - mean0


def plot_setup_helper(ax, scale=1):
    """Create map background for plotting spatial data.
    Returns modified 'ax' object."""

    ## specify range and ticklabels for plot
    lon_range = [-70, 10]
    lat_range = [3, 70]
    xticks = [-60, -40, -20, 0]
    yticks = [20, 40, 60]

    ax, gl = src.utils.plot_setup(ax, lon_range, lat_range, xticks, yticks, scale)

    return ax, gl

#### Data loading functions

In [ ]:
def load_noaa_uv10():
    """Load NOAA U10 and V10 data (eastward and northward windspeed at 10m)"""

    ## load raw files for u and v
    u10_noaa = xr.open_dataset(
        os.path.join(fp_uv10_noaa, "uwnd.10m", "monthly", "uwnd.10m.mon.mean.nc")
    )
    v10_noaa = xr.open_dataset(
        os.path.join(fp_uv10_noaa, "vwnd.10m", "monthly", "vwnd.10m.mon.mean.nc")
    )

    ## merge into single dataset
    uv10_noaa = xr.merge([u10_noaa, v10_noaa])

    return uv10_noaa


def load_noaa_precip():
    """Load NOAA precipitation data from CMIP server.
    Returns xr.dataarray with units of mm"""

    ## load in precipitation rate (units: kg/m^2/s)
    precip_rate = xr.open_dataset(fp_precip_noaa)["prate"]

    ## change units to mm (millimeters)
    precip = update_precip_units(precip_rate).rename("precip")

    return precip


def update_precip_units(precip_rate):
    """change units of precipitation from (kg m^-2 s^-1) to mm
    by dividing by density of water and integrating in time"""

    ## Convert units from mass/area to height by dividing
    ## by density of liquid water:
    ## (km m^-2 s^-1) * (kg^-1 m^3) = m s^-1
    density_water = 1000  # units: kg m^-3
    precip_rate = precip_rate / density_water  # new units of m/s

    ## convert from month-averaged rate (units: m/s) to
    ## month total (units: m), by multiplyin by no. of seconds
    ## in each month. Units: (m s^-1) * s = m.
    ## Note: days_per_month depends on month (so is
    ## array, not a scalar, in the code below).
    days_per_month = precip_rate.time.dt.days_in_month
    seconds_per_day = 86400
    seconds_per_month = seconds_per_day * days_per_month
    precip = precip_rate * seconds_per_month

    ## convert units from m to mm
    mm_per_m = 1000
    precip = precip * mm_per_m

    return precip


def load_noaa_lsm():
    """Load NOAA land-sea mask for NOAA reanalysis"""

    ## load raw data
    url = r"http://psl.noaa.gov/thredds/dodsC/Datasets/20thC_ReanV2c/gaussian/time_invariant/land.nc"
    lsm = xr.open_dataset(url).isel(time=0, drop=True).compute()

    ## remove this attribute: causes issues with saving to netcdf
    del lsm.attrs["_NCProperties"]

    return lsm


def update_coords_noaa(data):
    """function to update coordinates of NOAA data"""

    data = src.utils.switch_longitude_range(data)
    data = src.utils.reverse_latitude(data)

    return data


def trim_noaa(data, djf_avg=True):
    """function to trim non-SLP NOAA data"""

    ## update coordinates before trimming
    data = update_coords_noaa(data)

    ## Get DJF average if variable is a function of time
    if "time" in data.coords:
        data = src.utils.djf_avg(data).compute()

    ## trim in space
    data = src.utils_azores.trim_to_north_atlantic(data)

    return data


def get_trimmed_data_noaa(data_loader_fn, fp_out):
    return load_prepped_data(
        data_loader_fn=data_loader_fn, prep_fn=trim_noaa, fp_out=fp_out
    )

#### Do the actual data loading

In [ ]:
uv10_noaa = get_trimmed_data_noaa(
    load_noaa_uv10, fp_out=os.path.join(DATA_FP, "uv10_noaa.nc")
)
precip_noaa = get_trimmed_data_noaa(
    load_noaa_precip, fp_out=os.path.join(DATA_FP, "precip_noaa.nc")
)
lsm_noaa = get_trimmed_data_noaa(
    load_noaa_lsm, fp_out=os.path.join(DATA_FP, "lsm_noaa.nc")
)

## update grid of SLP so we can compare it to other variables
slp_noaa_regrid = slp_noaa.interp(
    {"lon": lsm_noaa.lon, "lat": lsm_noaa.lat}, method="linear"
)

## merge variables into single dataset
data_noaa = xr.merge([slp_noaa_regrid, uv10_noaa, precip_noaa])

#### Compute composite and climatology

In [ ]:
## compute composite and climatology
comp = make_composite(data_noaa)
clim = data_noaa.sel(year=slice(1950, 2007)).mean("year")

#### Plotting functions

In [ ]:
## functions to plot individual variables on plotting background.
## these functions take in an 'ax' object
def plot_precip(fig, ax):
    """plot precipitation composite"""

    ## plot data
    precip_plot = ax.contourf(
        comp.lon,
        comp.lat,
        comp["precip"],
        cmap="cmo.diff_r",
        levels=src.utils.make_cb_range(35, 3.5),
        extend="both",
    )

    ## add colorbar
    precip_cb = fig.colorbar(
        precip_plot,
        pad=0.05,
        ax=ax,
        orientation="horizontal",
        label=r"$\Delta$ Precip. (mm/month)",
        ticks=np.arange(-28, 42, 14),
    )

    return


def plot_slp(fig, ax, mask=False, add_colorbar=False):
    """Plot SLP. If mask==True, mask values over land"""

    ## mask data if specified
    if mask:
        plot_data = comp["slp"].where(lsm_noaa["land"] < 1)
    else:
        plot_data = comp["slp"]

    slp_plot = ax.contourf(
        comp.lon,
        comp.lat,
        plot_data,
        cmap="cmo.balance",
        levels=src.utils.make_cb_range(500, 50),
        transform=ccrs.PlateCarree(),
    )

    ## add colorbar if desired
    if add_colorbar:
        slp_cb = fig.colorbar(
            slp_plot,
            ax=ax,
            pad=0.05,
            orientation="horizontal",
            label=r"$\Delta$ SLP (Pa)",
            ticks=np.arange(-400, 600, 200),
        )

    return


def plot_slp_clim(fig, ax):
    """Plot SLP. If mask==True, mask values over land"""

    slp_plot_clim = ax.contour(
        clim.lon,
        clim.lat,
        clim["slp"],
        colors="k",
        levels=np.arange(99600, 102400, 400),
        linewidths=0.8,
    )

    return


def plot_wind(fig, ax, n=3):
    """Plot low-level wind. Plot every 'n'th vector to avoid overcrowding"""

    # get grid for plotting
    xx, yy = np.meshgrid(comp.lon.values[::n], comp.lat.values[::n])

    # plot the vectors
    wind_plot = ax.quiver(
        xx, yy, comp["uwnd"].values[::n, ::n], comp["vwnd"].values[::n, ::n]
    )

    # add legend
    wind_legend = ax.quiverkey(wind_plot, X=1.07, Y=-0.28, U=2, label=r"2 $m/s$")

    return

#### Make plot

In [ ]:
## Create figure
fig = plt.figure(figsize=(10, 6))

## add first plotting background
ax0 = fig.add_subplot(1, 2, 1, projection=ccrs.PlateCarree())
ax0, gl0 = plot_setup_helper(ax0, scale=1.2)

## plot precip in colors over land:
plot_precip(fig, ax0)
plot_slp(fig, ax0, mask=True)
plot_wind(fig, ax0, n=3)

## add second plotting background, and remove left longitude labels
ax1 = fig.add_subplot(1, 2, 2, projection=ccrs.PlateCarree())
ax1, gl1 = plot_setup_helper(ax1, scale=1.2)
gl1.left_labels = False

## for SLP: plot composite in colors and climatology in contours
plot_slp(fig, ax1, add_colorbar=True)
plot_slp_clim(fig, ax1)

plt.show()